Baseball Prediction: 1 - Data Wrangling
In this notebook we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building. Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file. This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html Towards the bottom of the page there are links for ZIP files containing multiple seasons. Download the 5 zip files for 1980-1989, 1990-1999, ... , 2020-2022, decompress them, and then move all of the single season files to a single directory. You will then need the path to that directory for the variable fname below.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
# Fill in the path to your data here...
fname = '/Users/willzinkann/Library/CloudStorage/OneDrive-Personal/Active/Baseball_Pred/game_data/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [4]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

In [5]:
df.columns = colnames

In [6]:
df.sample(10)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
1081,20220626,0,Sun,TOR,AL,72,MIL,NL,75,3,10,51,D,NaN,NaN,NaN,MIL06,35503.0,171,300000000,35000200x,31,6,0,0,1,3,0,0,0,3,0,6,0,0,2,0,4,5,10,10,0,0,24,12,0,0,1,0,34,13,3,0,2,10,0,1,1,4,0,5,2,0,1,0,6,5,3,3,0,0,27,12,0,0,2,0,barrt901,Ted Barrett,ballb901,Brock Ballou,viscj901,Jansen Visconti,hicke901,Ed Hickox,NaN,(none),NaN,(none),montc001,Charlie Montoyo,counc001,Craig Counsell,gustj001,Jandel Gustave,berrj001,Jose Berrios,NaN,(none),davij007,Jonathan Davis,berrj001,Jose Berrios,gonzc002,Chi Chi Gonzalez,sprig001,George Springer,8,bichb001,Bo Bichette,6,guerv002,Vladimir Guerrero,10,kirka001,Alejandro Kirk,2,hernt002,Teoscar Hernandez,9,espis001,Santiago Espinal,4,chapm001,Matt Chapman,5,gurrl001,Lourdes Gurriel,7,biggc002,Cavan Biggio,3,yelic001,Christian Yelich,10,adamw002,Willy Adames,6,tellr001,Rowdy Tellez,3,mccua001,Andrew McCutchen,7,urial001,Luis Urias,4,narvo001,Omar Narvaez,2,taylt002,Tyrone Taylor,9,petej002,Jace Peterson,5,davij007,Jonathan Davis,8,NaN,Y
2397,20221004,0,Tue,CHN,NL,161,CIN,NL,161,2,3,51,N,NaN,NaN,NaN,CIN09,13738.0,172,000100100,000000201,32,3,0,0,2,2,0,0,1,2,0,12,2,0,0,0,6,3,3,3,0,0,24,9,0,0,1,0,29,6,1,0,1,3,0,0,0,4,0,9,0,0,1,0,6,5,2,2,0,0,27,7,2,0,0,0,cuzzp901,Phil Cuzzi,willr901,Ryan Wills,hallt901,Tom Hallion,tosia901,Alex Tosi,NaN,(none),NaN,(none),rossd001,David Ross,belld002,David Bell,diaza004,Alexis Diaz,hughb002,Brandon Hughes,NaN,(none),stees001,Spencer Steer,assaj001,Javier Assad,cessl001,Luis Cessa,quire001,Esteban Quiroz,4,contw001,Willson Contreras,2,suzus001,Seiya Suzuki,9,happi001,Ian Happ,7,hoern001,Nico Hoerner,6,reyef001,Franmil Reyes,10,wisdp001,Patrick Wisdom,5,rivaa001,Alfonso Rivas,3,morec001,Christopher Morel,8,indij001,Jonathan India,4,fralj001,Jake Fraley,9,farmk001,Kyle Farmer,10,solad001,Donovan Solano,3,fairs001,Stuart Fairchild,7,stees001,Spencer Steer,5,sianm001,Mike Siani,8,garcj007,Jose Garcia,6,robic003,Chuckie Robinson,2,NaN,Y
366,20220504,2,Wed,PIT,NL,24,DET,AL,23,7,2,54,D,NaN,NaN,NaN,DET05,15150.0,187,001100203,000020000,34,11,4,0,2,5,0,1,1,5,0,3,1,1,2,0,7,4,2,2,0,0,27,11,2,0,0,0,37,8,4,0,0,2,0,0,0,2,1,9,0,0,0,0,10,4,7,7,1,0,27,9,1,0,2,0,becka901,Adam Beck,carlm901,Mark Carlson,addir901,Ryan Additon,ramoc901,Charlie Ramos,NaN,(none),NaN,(none),sheld801,Derek Shelton,hinca001,A.J. Hinch,thomz001,Zach Thompson,hutcd001,Drew

In [7]:
## Table with every game since 1990
df = pd.DataFrame()
for year in range(1990,2023):
    fname = '/Users/willzinkann/Library/CloudStorage/OneDrive-Personal/Active/Baseball_Pred/game_data/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))

In [8]:
df.shape

(75939, 162)

In [9]:
df.info(max_cols=100)

<class 'pandas.core.frame.DataFrame'>
Index: 75939 entries, 0 to 2429
Columns: 162 entries, date to season
dtypes: float64(1), int64(83), object(78)
memory usage: 94.4+ MB


In [10]:
## Calculate a few useful columns
df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)

In [11]:
# Basic Exploration
df.home_victory.mean()

0.5377605709846061

Big picture - have outcome, need features
Would like to start simple, base features on team results over past n games
Need to do a bit of "data wrangling"
First, we need to have each team's games in chronological order
Then can use pandas 'rolling' functionality to aggregate over windows in the past
Immediate goal, get batting average, on-base percentage, and slugging percentage
Need "running values" based on the past, not including the current game

Single games

In [12]:
df_red_sox = df.loc[((df.team_v=='BOS') | (df.team_h=='BOS')), :]

In [13]:
df_red_sox.shape

(5178, 166)

In [14]:
df_red_sox.head(100)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
0,19900409,0,Mon,DET,AL,1,BOS,AL,1,2,5,51,D,NaN,NaN,NaN,BOS07,35199.0,167,000002000,31000010x,33,5,1,0,0,2,0,1,0,4,0,6,0,0,0,0,9,3,2,2,0,0,24,8,2,0,0,0,33,8,4,0,0,4,0,0,0,4,2,3,0,0,0,0,8,3,2,2,0,0,27,11,2,0,0,0,barnl901,Larry Barnett,koscg901,Greg Kosc,roe-r901,Rocky Roe,morrd901,Dan Morrison,NaN,(none),NaN,(none),andes101,Sparky Anderson,morgj101,Joe Morgan,clemr001,Roger Clemens,morrj001,Jack Morris,smitl001,Lee Smith,evand002,Dwight Evans,morrj001,Jack Morris,clemr001,Roger Clemens,philt001,Tony Phillips,5,trama001,Alan Trammell,6,whitl001,Lou Whitaker,4,fielc001,Cecil Fielder,3,mosel001,Lloyd Moseby,8,nokem001,Matt Nokes,2,wardg001,Gary Ward,7,lemoc001,Chet Lemon,9,bergd001,Dave Bergman,10,boggw001,Wade Boggs,5,barrm001,Marty Barrett,4,greem001,Mike Greenwell,7,burke001,Ellis Burks,8,evand002,Dwight Evans,10,robib001,Billy Jo Robidoux,3,penat001,Tony Pena,2,reedj003,Jody Reed,6,romik001,Kevin Romine,9,NaN,Y,1990,3,1,7,199004090
10,19900410,0,Tue,DET,AL,2,BOS,AL,2,2,4,51,D,NaN,NaN,NaN,BOS07,15352.0,174,000020000,00400000x,34,7,1,0,0,2,0,0,1,1,0,7,0,0,0,0,7,3,4,4,2,0,24,13,0,0,2,0,31,10,2,0,0,4,1,0,0,8,4,3,1,0,2,0,12,4,2,2,0,0,27,6,0,0,0,0,koscg901,Greg Kosc,roe-r901,Rocky Roe,morrd901,Dan Morrison,barnl901,Larry Barnett,NaN,(none),NaN,(none),andes101,Sparky Anderson,morgj101,Joe Morgan,boddm001,Mike Boddicker,ritzk001,Kevin Ritz,smitl001,Lee Smith,penat001,Tony Pena,ritzk001,Kevin Ritz,boddm001,Mike Boddicker,philt001,Tony Phillips,5,trama001,Alan Trammell,6,whitl001,Lou Whitaker,4,fielc001,Cecil Fielder,3,mosel001,Lloyd Moseby,8,nokem001,Matt Nokes,2,sheel001,Larry Sheets,7,lemoc001,Chet Lemon,9,bergd001,Dave Bergman,10,boggw001,Wade Boggs,5,barrm001,Marty Barrett,4,greem001,Mike Greenwell,7,burke001,Ellis Burks,8,evand002,Dwight Evans,10,robib001,Billy Jo Robidoux,3,penat001,Tony Pena,2,reedj003,Jody Reed,6,romik001,Kevin Romine,9,NaN,Y,1990,2,1,6,199004100
19,19900411,0,Wed,DET,AL,3,BOS,AL,3,2,3,59,D,NaN,NaN,NaN,BOS07,15369.0,204,0000011000,0000101001,36,7,1,0,0,2,1,0,0,1,0,7,0,0,1,0,6,5,3,3,0,0,29,15,0,0,1,0,36,12,3,0,1,3,1,0,1,3,0,6,0,3,1,0,9,4,1,1,1,0,30,14,1,0,1,0,roe-r901,Rocky Roe,morrd901,Dan Morrison,barnl901,Larry Barnett,koscg901,Greg Kosc,NaN,(none),NaN,(none),andes101,Sparky Anderson,morgj101,Joe Morgan,harrg001,Gre

In [15]:
# Write a function to create a team-specific data frame, given the team
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)

In [16]:
df_red_sox = create_team_df('BOS')

In [17]:
df_red_sox.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003012,20030413,0,Sun,BOS,AL,12,2,51,D,NaN,NaN,NaN,BOS07,32368.0,150,10000010x,31,6,2,0,1,2,0,0,0,1,0,5,2,0,0,0,6,2,0,0,0,0,27,12,0,0,2,0,gibsg901,Greg Gibson,iassd901,Dan Iassogna,demud901,Dana DeMuth,hudsm901,Marvin Hudson,NaN,(none),NaN,(none),littg801,Grady Little,lowed001,Derek Lowe,loper001,Rodrigo Lopez,waket001,Tim Wakefield,garcn001,Nomar Garciaparra,lowed001,Derek Lowe,damoj001,Johnny Damon,8,walkt002,Todd Walker,4,garcn001,Nomar Garciaparra,6,ramim002,Manny Ramirez,7,millk005,Kevin Millar,10,hills002,Shea Hillenbrand,3,nixot001,Trot Nixon,9,muelb001,Bill Mueller,5,varij001,Jason Varitek,2,NaN,Y,2003,2,1,2,200304130,1,BAL,5681.0,1566.0,367.0,33.0,176.0,554.0,884.0,83.0,27.0,103.0,0.275656,0.340016,0.444816,0.784832,1075.0,279.0,76.0,4.0,33.0,129.0,173.0,12.0,6.0,19.0,0.259535,0.338870,0.429767,0.768638
2002081,20020703,0,Wed,BOS,AL,81,5,51,N,NaN,NaN,NaN,BOS07,31777.0,184,00100103x,34,11,0,0,0,4,0,0,1,4,0,7,5,0,1,0,10,3,2,2,0,0,27,5,1,0,2,0,culbf901,Fieldin Culbreth,millb901,Bill Miller,hohnb901,Bill Hohn,younl901,Larry Young,NaN,(none),NaN,(none),littg801,Grady Little,gomew001,Wayne Gomes,polic001,Cliff Politte,urbiu001,Ugueth Urbina,clart002,Tony Clark,burkj001,John Burkett,damoj001,Johnny Damon,10,hills002,Shea Hillenbrand,5,garcn001,Nomar Garciaparra,6,ramim002,Manny Ramirez,7,daubb001,Brian Daubach,9,varij001,Jason Varitek,2,clart002,Tony Clark,3,nixot001,Trot Nixon,8,merll001,Lou Merloni,4,NaN,Y,2002,3,1,7,200207030,1,TOR,5593.0,1494.0,307.0,30.0,180.0,531.0,793.0,73.0,45.0,108.0,0.267120,0.330666,0.429287,0.759953,1008.0,252.0,55.0,9.0,22.0,88.0,129.0,20.0,5.0,17.0,0.250000,0.310219,0.387897,0.698116
1993065,19930617,0,Thu,BOS,AL,65,0,51,N,NaN,NaN,NaN,TOR02,50528.0,169,000000000,28,2,0,0,0,0,0,0,0,2,0,6,0,0,1,0,3,3,7,7,0,0,24,8,0,0,1,0,craft901,Terry Craft,evanj901,Jim Evans,mccol901,Larry McCoy,hendt901,Ted Hendry,NaN,(none),NaN,(none),hobsb101,Butch Hobson,leita001,Al Leiter,quanp001,Paul Quantrill,NaN,(none),butlr001,Rob Butler,quanp001,Paul Quantrill,hatcb001,Billy Hatcher,8,caldi001,Ivan Calderon,9,greem001,Mike Greenwell,7,vaugm001,Mo Vaughn,10,quinc001,Carlos Quintana,3,melvb001,Bob Melvin,2,coops001,Scott Cooper,5,valej002,John Valentin,6,rivel001,Luis Rivera,4,NaN,Y,1993,7,1,7,199306170,0,TOR,5460.0,1381.0,269.0,29.0,84.0,531.0,621.0,50.0,42.0,150.0,0.252930,0.319145,0.358974,0.678120,997.0,256.0,58.0,6.0,14.0,119.0,128.0,18.0,8.0,26.0,0.256770,0.336022,0.369107,0.705129
2010119,20100815,

In [18]:
# Make a dictionary that maps a team name to it's data frame

# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [19]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features


## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%1000==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000


In [20]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v

In [21]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
1924,19920920,0,Sun,BOS,AL,149,DET,AL,149,5,4,54,D,NaN,NaN,NaN,DET04,18459.0,186,000110120,101000002,37,12,1,1,2,5,0,0,0,2,0,1,0,0,1,0,7,4,4,4,1,0,27,8,0,0,0,0,35,8,3,0,0,3,0,0,0,8,0,6,0,0,0,0,12,2,5,5,1,0,27,13,1,0,1,0,reilm901,Mike Reilly,cousd901,Derryl Cousins,NaN,(none),brinj901,Joe Brinkman,NaN,(none),NaN,(none),hobsb101,Butch Hobson,andes101,Sparky Anderson,violf001,Frank Viola,gullb001,Bill Gullickson,fosst001,Tony Fossas,valej002,John Valentin,violf001,Frank Viola,gullb001,Bill Gullickson,hatcb001,Billy Hatcher,7,penat001,Tony Pena,2,coops001,Scott Cooper,5,brunt001,Tom Brunansky,9,planp001,Phil Plantier,10,zupcb001,Bob Zupcic,8,lyons001,Steve Lyons,3,valej002,John Valentin,6,naeht001,Tim Naehring,4,philt001,Tony Phillips,4,gladd001,Dan Gladden,8,frymt001,Travis Fryman,6,fielc001,Cecil Fielder,3,deerr001,Rob Deer,9,carrm001,Mark Carreon,7,barns001,Skeeter Barnes,5,rowlr001,Rich Rowland,10,kreuc001,Chad Kreuter,2,NaN,Y,1992,-1,0,9,199209200,0.258234,0.246977,0.337910,0.319987,0.408795,0.346097,0.746705,0.666084,74.0,44.0,50.0,49.0,117.0,140.0,0.257200,0.252896,0.335702,0.321053,0.396226,0.345560,0.731928,0.666612,15.0,4.0,10.0,10.0,21.0,33.0
1771,20010814,0,Tue,MON,NL,119,LAN,NL,119,4,1,54,N,NaN,NaN,NaN,LOS03,30085.0,198,000000004,010000000,32,7,3,1,0,4,0,0,0,4,2,4,0,0,1,0,5,5,1,1,2,0,27,10,0,0,3,0,29,5,0,0,1,1,0,0,1,10,1,6,0,0,3,0,12,2,4,4,0,0,27,13,0,0,1,0,wendh902,Hunter Wendelstedt,wolfj901,Jim Wolf,herna901,Angel Hernandez,marsr901,Randy Marsh,NaN,(none),NaN,(none),torbj101,Jeff Torborg,tracj101,Jim Tracy,lloyg001,Graeme Lloyd,shawj001,Jeff Shaw,NaN,(none),cabro001,Orlando Cabrera,armat002,Tony Armas,parkc002,Chan Ho Park,bergp001,Peter Bergeron,8,blumg001,Geoff Blum,5,guerv001,Vladimir Guerrero,9,stevl001,Lee Stevens,3,cabro001,Orlando Cabrera,6,smitm004,Mark Smith,7,knorr001,Randy Knorr,2,mordm001,Mike Mordecai,4,armat002,Tony Armas,1,coraa001,Alex Cora,6,grudm001,Mark Grudzielanek,4,grees001,Shawn Green,9,shefg001,Gary Sheffield,7,karre001,Eric Karros,3

In [22]:
df.to_csv('df_bp1.csv', index=False)